In [1]:
import gradio as gr
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import io
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

ENDPOINT = "https://team10eighticustomvision-prediction.cognitiveservices.azure.com/"
PREDICTION_KEY = "9FRZbiwBubFIcSZ1k88tCTCskOAZwMMAMvnFLuVJ26tlU0V0fsqjJQQJ99ALACYeBjFXJ3w3AAAIACOGOj1S"
PROJECT_ID = "4218ecac-688a-422b-9e14-2726b938f67c"
PUBLISHED_NAME = "Iteration8"
 
# Initialize the prediction client
credentials = ApiKeyCredentials(in_headers={"Prediction-key": PREDICTION_KEY})
predictor = CustomVisionPredictionClient(endpoint=ENDPOINT, credentials=credentials)

def draw_boxes(image, predictions):
    """가장 확률이 높은 객체만 경계 상자를 그리도록 수정"""
    img = image.copy()
    
    # 예측 결과 중 확률이 가장 높은 하나를 선택
    if predictions:
        highest_prediction = max(predictions, key=lambda p: p.probability)
        
        # 확률이 0.5 이상인 객체만 선택
        if highest_prediction.probability > 0.5:
            color = (255, 0, 0)  # 경계 상자 색상 (빨간색)
            box = highest_prediction.bounding_box
            left = int(box.left * img.shape[1])
            top = int(box.top * img.shape[0])
            width = int(box.width * img.shape[1])
            height = int(box.height * img.shape[0])

            # 경계 상자 그리기
            cv2.rectangle(img, 
                        (left, top), 
                        (left + width, top + height), 
                        color, 
                        2)
            
            # 라벨과 확률 텍스트 추가
            label = f"{highest_prediction.tag_name}: {highest_prediction.probability:.2f}"
            cv2.putText(img, 
                        label, 
                        (left, top - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, 
                        color, 
                        2)
    
    return img


def process_frame(frame):
    if frame is None:
        return None
    
    # Convert frame to PIL Image
    pil_image = Image.fromarray(frame)
    
    # Save to bytes for Azure API
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()

    try:
        # Get predictions from Azure Custom Vision
        results = predictor.detect_image(PROJECT_ID, PUBLISHED_NAME, img_byte_arr)
        
        # Draw boxes on frame
        annotated_frame = draw_boxes(frame, results.predictions)
        
        return annotated_frame
    
    except Exception as e:
        print(f"Error during prediction: {e}")
        return frame


# Blocks section
with gr.Blocks() as demo:
    with gr.Row():  # Row layout for input and output
        with gr.Column():  # Input webcam column
            webcam_input = gr.Image(sources="webcam", streaming=True, mirror_webcam=True, label="Webcam")
        with gr.Column():  # Output display column
            output = gr.Image(label="Detected Objects")

    # Set the process_frame function as the update function for webcam input
    webcam_input.stream(process_frame, inputs=webcam_input, outputs=output)
    
    demo.title = "Azure Custom Vision Object Detection"
    demo.description = "Real-time object detection using Azure Custom Vision"

if __name__ == "__main__":
    demo.launch(share=True)


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://2b4efff59b5468ef4f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
